# convert structured JSON API information into OpenAgent plugins

In [ ]:
from openai import OpenAI
import os
import uuid
import json
import re

In [ ]:
client = OpenAI(api_key=input("your-api-key?"))

In [ ]:
target_dir = "../extractor/apidocs"
plugin_dir = "pluginpath"
register_file = "filepath"

In [ ]:
def generate_name_and_desc_json(json):
    completion = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant skilled in summarizing json content."},
            {"role": "user", "content": "You will be provided with a json file for API info. Summarize and then provide the description and name of this API for me. The output format is: Desc: {your output}\nName: {your output}"},
            {"role": "user", "content": json}
        ],
        temperature=0,
    )
    return completion.choices[0].message.content

In [ ]:
def create_plugin_folder(plugin_dir, plugin_name):
    new_plugin_path = os.path.join(plugin_dir, plugin_name)
    os.makedirs(new_plugin_path, exist_ok=True)
    print(f"Folder '{plugin_name}' created in '{plugin_dir}'")
    return new_plugin_path

def create_init_py(path, content):
    init_file_path = os.path.join(path, '__init__.py')
    with open(init_file_path, 'w') as f:
        if content == None:
            pass
        else:
            f.write(content)
    print(f"'__init__.py' created in '{path}'")

def create_ai_plugin_json(path,id, domain, namespace, desc, yaml_url):
    with open('../template/ai_plugin_json_template.txt', 'r') as template_file:
        json_template = template_file.read()
    ai_plugin_json = json_template.format(id=id, domain=domain, namespace=namespace, desc=desc, yaml_url=yaml_url)
    json_file_path = os.path.join(path, 'ai-plugin.json')
    with open(json_file_path, 'w') as f:
        f.write(ai_plugin_json)
    print(f"'ai-plugin.json' created in '{path}'")

def generate_plugin_id():
    unique_id = uuid.uuid4()
    return f"plugin-{str(unique_id)}"

In [ ]:
for folder in os.listdir(target_dir):
    
    # create plugin folder
    new_plugin_path = create_plugin_folder(plugin_dir, folder)
    
    # create init.py within the folder
    create_init_py(new_plugin_path, None)
    
    # create ai-plugin.json within the folder
    tool_path = os.path.join(target_dir, folder)
    try:
        for tool_json in os.listdir(tool_path):
            if tool_json.endswith(".txt"):
                with open(os.path.join(tool_path, tool_json), "r") as f:
                    json_text = f.read()
                    json_content = json.loads(json_text)
                domain = json_content["endpoints"][0]["url"].split("/")[2]
                yaml_url = "https://" + domain + "/openapi.yaml"
                gpt_answer = generate_name_and_desc_json(json_text)
                namespace = gpt_answer.split("\n")[1][6:]
                desc = gpt_answer.split("\n")[0][6:]        
                id = generate_plugin_id()
                create_ai_plugin_json(new_plugin_path, id, domain, namespace, desc, yaml_url)
    except:
        print(folder + "!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\n\n\n\n")
        continue
    
    # create /path
    path_dir = new_plugin_path + "/path"
    os.makedirs(path_dir, exist_ok=True)

    path_dict = {}
    for tool_function in os.listdir(tool_path):
        request = ""
        if tool_function.endswith(".py"):
            with open(os.path.join(tool_path, tool_function), "r") as f:
                code = f.read()
            api_url = re.search(r"api_url = .+", code).group()
            request += api_url + "\n"
            is_get_used = re.search(r"requests\.get", code) is not None
            is_post_used = re.search(r"requests\.post", code) is not None
            if is_get_used:
                request += f'    response = requests.get(api_url, params=input_json)'
            if is_post_used:
                request += f'    response = requests.post(api_url, json=input_json)'

            with open('../template/call_api_template.txt', 'r') as template_file:
                template = template_file.read()
            template = template.format(request=request)
            
            with open(os.path.join(path_dir, tool_function), "w") as f:
                f.write(template)

            position = -1
            for _ in range(3):
                position = api_url.find('/', position + 1)
            
            path_dict[tool_function] = api_url[position:]

    create_init_py(path_dir, "path_dict = " + str(path_dict))
    
    # register this plugin
    variable = folder.upper()
    with open(register_file, 'a') as f:
        f.write(f'    {variable} = "{folder}"\n')
    print(f'{folder} added as a plugin\n\n')